In [1]:
from datascience import *
import numpy as np
from math import *
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline

## Lesson 32: Likelihood Ratio Tests

Last time, we introduced Likelihood Ratio tests. Recall that the point of a likelihood ratio test is to compare the likelihood function under a hypothesized value of the parameter with the liklihood function at its maximum. Instead of looking at the ratio $\Lambda$ itself, we often consider $-2\log \Lambda$ instead, since it has a handy distribution. 

### Example 1: Exponential Distribution

Suppose $X_1,X_2,...,X_n$ is an iid sequence of random variables from the exponential distribution with unknown parameter $\lambda$. Recall that the maximum likelihood estimate of $\lambda$ is $1\over\bar{X}$. We collect a random sample of size 20 and want to test the hypothesis $H_0: \lambda = 3$ vs $H_1: \lambda \neq 3$. Using the data in the python box below, conduct a likelihood ratio test on this hypothesis.  

In [2]:
my_data=np.array([0.18,0.277,0.105,0.126,0.225,0.026,0.123,0.423,0.006,0.281,0.050,0.692,0.105,0.275,0.346,0.079,0.045,0.222,0.063,0.281])

$$
\Lambda=\frac{L(\lambda\mid\textbf{x})}{L(\hat{\lambda}_{ML}\mid\textbf{x})}
$$
$$
-2\log \Lambda = 2\left[n \log \hat{\lambda}_{ML} - \hat{\lambda}_{ML} \sum x_i-n \log \lambda_0 - \lambda_0 \sum x_i\right] 
$$
$$
-2\log \Lambda = 2\left[n \log \hat{\lambda}_{ML} - \hat{\lambda}_{ML} n \bar{X}- n \log \lambda_0 + n \right] 
$$

In [3]:
lam_not = 1/np.mean(my_data)
test_stat = -2*(20*np.log(3)-20*3*np.mean(my_data)-20*np.log(lam_not)+20)
test_stat

4.7192223601884535

In [4]:
1-stats.chi2.cdf(test_stat,1)

0.029827229194775318

#### Power

Suppose that the true value of $\lambda$ is 5. Let's determine the power of this test. Let $n=20$. Then determine the power if $n=50$. Remember, power is the probability of correctly rejecting the null hypothesis. 

First, find what value of $-2 \log \Lambda$ would lead you to reject $H_0$. This is sometimes called the critical value. 

In [24]:
crit = stats.chi2.ppf(.95,1)
crit

3.841458820694124

Next, obtain the power. Obtain a sample of size 20 from the true population and obtain the value of $-2\log \Lambda$ for this sample. Repeat many times and determine how often you reject the null hypothesis. 

In [25]:
def log_stat(data, null, n):
    lam_not = 1/np.mean(data)
    return -2*(n*np.log(null)-n*null*np.mean(data)-n*np.log(lam_not)+n)
    

In [26]:
log_stat(my_data, null=3, n=20)

4.7192223601884535

In [27]:
np.random.seed(500)
sims = 10000
test = make_array()
for i in np.arange(sims):
    x = stats.expon.rvs(scale=1/5, size=20)
    test = np.append(test, log_stat(x, 3, 20))

sample_20 = np.mean(test>=crit)
sample_20

0.5978

Repeat for a sample size of 50. What do you expect to happen to power? 

In [28]:
test2 = make_array()
for i in np.arange(sims):
    y = stats.expon.rvs(scale=1/5, size=50)
    test2 = np.append(test2, log_stat(y, 3, 50))

sample_50 = np.mean(test2>=crit)
sample_50

0.9525

### A Different Test

We've explored hypothesis tests in this class before. Taking advantage of our computing power, we don't have to rely on test statistics with asymptotic distributions. Let's conduct a more direct hypothesis test using simulation. Recall:

$$
H_0: \lambda = 3
$$

$$
H_1: \lambda \neq 3
$$

Pick a different test statistic. Obtain an empirical distribution of that test statistic under $H_0$. Next, find the $p$-value by determining how often this test statistic is at or further away from the test statistic derived from the sample. Remember that this is a two-sided test. 

In [32]:
test3 = make_array()
for i in np.arange(sims):
    z = np.mean(stats.expon.rvs(scale=1/3, size=20))
    test3 = np.append(test3, z)

sample3 = 2*np.mean(test3<=np.mean(my_data))
sample3

0.038

How did the $p$-value compare to the LRT $p$-value? I wonder how the power of this test compares to our LRT. 

#### Power

Let's figure out the power of this test. First, determine for what values of the test statistic would lead us to reject $H_0$. These values can be referred to as your rejection region. 

In [31]:
low = percentile(2.5,test3)
high = percentile(97.5,test3)
print(low, high)

0.20359198580601992 0.49241447042545083


Now, determine the power of this test. Like in the LRT case, obtain a sample of size 20 and obtain the test statistic. Repeat many times and see how often your test statistic is in your rejection region. 

In [33]:
test4 = make_array()
for i in np.arange(sims):
    a = np.mean(stats.expon.rvs(scale=1/5, size=20))
    test4 = np.append(test4, a)

sample4 = np.mean(test4<=low) + np.mean(test4>=high)
sample4

0.5584

Repeat for a sample size of 50. Note that you will have to obtain new critical values in order to do this.  

In [34]:
test3 = make_array()
for i in np.arange(sims):
    z = np.mean(stats.expon.rvs(scale=1/3, size=50))
    test3 = np.append(test3, z)

sample3 = 2*np.mean(test3<=np.mean(my_data))
sample3

low = percentile(2.5,test3)
high = percentile(97.5,test3)
print(low, high)

test4 = make_array()
for i in np.arange(sims):
    a = np.mean(stats.expon.rvs(scale=1/5, size=50))
    test4 = np.append(test4, a)

sample4 = np.mean(test4<=low) + np.mean(test4>=high)
sample4

0.247186851622122 0.4367263789776643


0.9437